<a href="https://colab.research.google.com/github/cssh22049/softwea/blob/main/SoftWea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 必要なパッケージをインストール
!pip install geopy networkx osmnx folium

import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
import folium
from IPython.display import display

class UserInterface:
    def __init__(self):
        self.route_planner = RoutePlanner()

    def start(self):
        current_location = input("現在地を入力してください（例：君津駅、千葉県、日本）：")
        destination = input("目的地を入力してください（例：船橋日大前駅、千葉県、日本）：")

        route_map = self.route_planner.calculate_route(current_location, destination)

        self.display_route(route_map)

    def display_route(self, route_map):
        if route_map:
            display(route_map)
        else:
            print("ルートが見つかりませんでした。")

class RoutePlanner:
    def __init__(self):
        self.geolocator = Nominatim(user_agent="route_planner")
        self.traffic_api = TrafficAPI()
        self.map_api = MapAPI()

    def calculate_route(self, current_location, destination):
        current_coords = self.get_location(current_location)
        destination_coords = self.get_location(destination)

        if current_coords and destination_coords:
            traffic_info = self.traffic_api.get_traffic_info(current_coords, destination_coords)
            route_map = self.map_api.get_shortest_route(current_coords, destination_coords, traffic_info)
            return route_map
        else:
            return None

    def get_location(self, address):
        location = self.geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            print(f"住所が見つかりませんでした: {address}")
            return None

class TrafficAPI:
    def __init__(self):
        self.api_key = "YOUR_TRAFFIC_API_KEY"  # ダミーAPIキー

    def get_traffic_info(self, current_coords, destination_coords):
        # ダミーの交通情報を返す
        return {
            "traffic_delay": 10  # ダミー値（分単位の遅延）
        }

class MapAPI:
    def __init__(self):
        pass

    def get_shortest_route(self, current_coords, destination_coords, traffic_info):
        try:
            # 使用する距離を大きくする
            G = ox.graph_from_point(current_coords, dist=20000, network_type='drive')
            orig_node = ox.nearest_nodes(G, current_coords[1], current_coords[0])
            dest_node = ox.nearest_nodes(G, destination_coords[1], destination_coords[0])

            # 最短ルートを計算する
            if nx.has_path(G, orig_node, dest_node):
                route = nx.shortest_path(G, orig_node, dest_node, weight='length')
            else:
                print("指定された場所の間にルートが存在しません。")
                return None

            # Foliumを使ってルートを表示
            route_map = folium.Map(location=[(current_coords[0] + destination_coords[0]) / 2,
                                             (current_coords[1] + destination_coords[1]) / 2], zoom_start=10)

            route_nodes = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
            folium.PolyLine(locations=route_nodes, color="blue", weight=2.5, opacity=1).add_to(route_map)

            # 現在地と目的地にマーカーを追加
            folium.Marker(location=[current_coords[0], current_coords[1]], popup="現在地", icon=folium.Icon(color='green')).add_to(route_map)
            folium.Marker(location=[destination_coords[0], destination_coords[1]], popup="目的地", icon=folium.Icon(color='red')).add_to(route_map)

            return route_map
        except Exception as e:
            print(f"ルート計算中にエラーが発生しました: {e}")
            return None

# メイン関数
ui = UserInterface()
ui.start()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 1.9 MB/s eta 0:00:00
現在地を入力してください（例：君津駅、千葉県、日本）：君津駅、千葉県、日本
目的地を入力してください（例：船橋日大前駅、千葉県、日本）：木更津駅、千葉県、日本
